# Initial Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(57)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "model_outputs", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Initial Data Imports

In [2]:
tictac_single = np.loadtxt('./datasets-part1/tictac_single.txt')
tictac_multi = np.loadtxt('./datasets-part1/tictac_multi.txt')
tictac_finals = np.loadtxt('./datasets-part1/tictac_final.txt')

In [3]:
# Transcribing the single tictac data:
xs, ys = tictac_single[:,:9], tictac_single[:,9:]
# used flatten to flatten y into a 1D array
ys = ys.flatten()
# x is input features and y is output labels (optimal move)

In [4]:
# Transcribing the multi tictac data:
xm, ym = tictac_multi[:,:9], tictac_multi[:,9:]
ym # not sure as whether I should flatten or not
# x is input features and y is output labels (optimal moves)

array([[0., 0., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [5]:
# Transcribing the final tictac data:
xf, yf = tictac_finals[:,:9], tictac_finals[:,9:]
# used flatten to flatten y into a 1D array
yf = yf.flatten()
# x is input features and y is output labels (who won the game)

# Models for Single Dataset

In [6]:
xs_train, xs_test, ys_train, ys_test = xs[:5241], xs[5241:], ys[:5241], ys[5241:]

In [7]:
ys_test[0:10]

array([5., 2., 4., 3., 0., 8., 0., 8., 6., 4.])

### First Model, Linear SVM

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm = Pipeline([
("scaler", StandardScaler()),
("linear_svc", LinearSVC(C=1, loss="hinge", dual="auto")),
])
svm.fit(xs_train, ys_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, dual='auto', loss='hinge'))])

In [9]:
svm.predict(xs_test[0:10])

array([8., 3., 3., 3., 3., 3., 3., 8., 3., 4.])

### Second Model, K-Nearest Neighbor

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=50) # 50 was picked as an arbitrary number of neighbors
knn.fit(xs_train, ys_train)

KNeighborsClassifier(n_neighbors=50)

In [11]:
knn.predict(xs_test[0:10])

array([4., 2., 4., 3., 0., 0., 0., 8., 6., 4.])

### Third Model, Multilayer Perceptron

In [12]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000)
mlp.fit(xs_train, ys_train)

MLPClassifier(max_iter=2000)

In [13]:
mlp.predict(xs_test[0:10])

array([5., 2., 4., 3., 0., 8., 0., 8., 6., 4.])

In [14]:
# Storing the models:
import pickle
with open('./model_outputs/models/svm_single.pickle', 'wb') as f:
    pickle.dump(svm, f)
with open('./model_outputs/models/knn_single.pickle', 'wb') as f:
    pickle.dump(knn, f)
with open('./model_outputs/models/mlp_single.pickle', 'wb') as f:
    pickle.dump(mlp, f)

# To Load
# with open('./model_outputs/models/mlp_single.pickle', 'rb') as f:
#    mlp = pickle.load(f)

### Reporting Performance on Single Tic Tac Models:

Measuring Accuracy using cross_val_score:

In [15]:
from sklearn.model_selection import cross_val_score
single_cvs_svm = cross_val_score(svm, xs_test, ys_test, cv=10, scoring="accuracy") # cv parameter sets the number of KFolds
np.mean(single_cvs_svm)

0.10687022900763359

In [16]:
single_cvs_knn = cross_val_score(knn, xs_test, ys_test, cv=10, scoring="accuracy")
np.mean(single_cvs_knn)

0.5625954198473282

In [17]:
single_cvs_mlp = cross_val_score(mlp, xs_test, ys_test, cv=10, scoring="accuracy")

In [18]:
np.mean(single_cvs_mlp)

0.8557251908396948

Generating normalized confusion matrix

In [19]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [20]:
# Perfect prediction
ys_perf = ys_test
confusion_matrix(ys_test, ys_perf)

array([[305,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 177,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 195,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  97,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 194,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,  69,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 103,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  53,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 117]], dtype=int64)

In [21]:
ys_pred = cross_val_predict(svm, xs_test, ys_test, cv=10)
confusion_matrix(ys_test, ys_pred)

array([[ 40,   3,  26,  38,  15,  13,  24, 119,  27],
       [ 15,   5,  14,  10,  15,   7,  12,  82,  17],
       [ 22,   2,  22,  21,  13,   9,  15,  76,  15],
       [ 11,   1,  12,  17,   7,   5,   6,  29,   9],
       [ 32,   3,  26,  15,  10,   5,  16,  74,  13],
       [ 12,   2,   8,   7,   4,   4,   6,  20,   6],
       [ 11,   2,  14,   8,  13,   5,   8,  31,  11],
       [  5,   0,   4,   6,   1,   3,   3,  27,   4],
       [ 12,   5,  11,  16,   7,  10,   9,  42,   5]], dtype=int64)

In [22]:
ys_pred = cross_val_predict(knn, xs_test, ys_test, cv=10)
confusion_matrix(ys_test, ys_pred)

array([[276,   3,  11,   1,  11,   0,   2,   0,   1],
       [ 24, 104,  14,   6,  22,   0,   2,   0,   5],
       [ 35,   8, 130,   2,  16,   0,   0,   0,   4],
       [ 29,   7,   6,  29,  18,   0,   4,   0,   4],
       [ 37,  13,  11,   2, 128,   0,   0,   0,   3],
       [ 18,   8,  16,   3,  11,   6,   3,   0,   4],
       [ 37,   6,  16,   3,  13,   0,  28,   0,   0],
       [ 23,   9,   6,   4,   8,   0,   0,   0,   3],
       [ 47,   7,  16,   4,   7,   0,   0,   0,  36]], dtype=int64)

In [23]:
ys_pred = cross_val_predict(mlp, xs_test, ys_test, cv=10)
confusion_matrix(ys_test, ys_pred)

array([[274,   3,   7,   5,   9,   0,   3,   2,   2],
       [  5, 155,   3,   2,   3,   4,   2,   1,   2],
       [  4,   2, 174,   2,   8,   1,   2,   0,   2],
       [  4,   5,   2,  70,   5,   2,   4,   3,   2],
       [  5,   4,   4,   2, 173,   3,   1,   2,   0],
       [  3,   5,   1,   3,   1,  51,   2,   2,   1],
       [  6,   3,   6,   4,   2,   1,  79,   1,   1],
       [  0,   4,   2,   1,   1,   0,   2,  42,   1],
       [  2,   4,   1,   0,   3,   4,   0,   0, 103]], dtype=int64)

# Models for Final Dataset

In [24]:
from sklearn.model_selection import train_test_split # learned about this towards the end of the assignment
xf_train, xf_test, yf_train, yf_test = train_test_split(xf, yf, test_size=0.3, random_state=0)

In [25]:
svm = Pipeline([
("scaler", StandardScaler()),
("linear_svc", LinearSVC(C=1, loss="hinge", dual="auto")),
])
svm.fit(xf_train, yf_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, dual='auto', loss='hinge'))])

In [26]:
yf_test[0:10]

array([-1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.])

In [27]:
svm.predict(xf_test[0:10])

array([-1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.])

### Second Model, K-Nearest Neighbor

In [28]:
knn = KNeighborsClassifier(n_neighbors=50) # 50 was picked as an arbitrary number of neighbors
knn.fit(xf_train, yf_train)

KNeighborsClassifier(n_neighbors=50)

In [29]:
knn.predict(xf_test[0:10])

array([ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.])

### Third Model, Multilayer Perceptron

In [30]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=2000)
mlp.fit(xf_train, yf_train)

MLPClassifier(max_iter=2000)

In [31]:
mlp.predict(xf_test[0:10])

array([-1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.])

### Reporting Performance on Final Tic Tac Models:

Measuring Accuracy using cross_val_score:

In [32]:
final_cvs_svm = cross_val_score(svm, xf_test, yf_test, cv=10, scoring="accuracy") # cv parameter sets the number of KFolds
np.mean(final_cvs_svm)

0.9616995073891627

In [33]:
final_cvs_knn = cross_val_score(knn, xf_test, yf_test, cv=10, scoring="accuracy")
np.mean(final_cvs_knn)

0.7118226600985221

In [34]:
final_cvs_mlp = cross_val_score(mlp, xf_test, yf_test, cv=10, scoring="accuracy")
final_cvs_mlp

array([1.        , 1.        , 1.        , 1.        , 0.93103448,
       1.        , 0.93103448, 0.96551724, 0.89285714, 1.        ])

In [35]:
np.mean(final_cvs_mlp)

0.9720443349753694

Generating normalized confusion matrix

In [36]:
# Perfect prediction
ys_perf = ys_test
confusion_matrix(ys_test, ys_perf)

array([[305,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 177,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 195,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  97,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 194,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,  69,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 103,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  53,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 117]], dtype=int64)

In [37]:
yf_pred = cross_val_predict(svm, xf_test, yf_test, cv=10)
confusion_matrix(yf_test, yf_pred)

array([[ 89,  11],
       [  0, 188]], dtype=int64)

In [38]:
yf_pred = cross_val_predict(knn, xf_test, yf_test, cv=10)
confusion_matrix(yf_test, yf_pred)

array([[ 18,  82],
       [  1, 187]], dtype=int64)

In [39]:
yf_pred = cross_val_predict(mlp, xf_test, yf_test, cv=10)
confusion_matrix(yf_test, yf_pred)

array([[ 90,  10],
       [  1, 187]], dtype=int64)

In [40]:
# Storing the models:
import pickle
with open('./model_outputs/models/svm_final.pickle', 'wb') as f:
    pickle.dump(svm, f)
with open('./model_outputs/models/knn_final.pickle', 'wb') as f:
    pickle.dump(knn, f)
with open('./model_outputs/models/mlp_final.pickle', 'wb') as f:
    pickle.dump(mlp, f)

# To Load
# with open('./model_outputs/models/mlp_single.pickle', 'rb') as f:
#    mlp = pickle.load(f)

## Models for Intermediate Dataset

In [41]:
xm_train, xm_test, ym_train, ym_test =  train_test_split(xm, ym, test_size=0.2, random_state=0)

### Model 1: KNN Regressor

In [42]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor(n_neighbors=50) # picked 50 neighbors for consistency with the other KNN models
knn_reg.fit(xm_train, ym_train)

KNeighborsRegressor(n_neighbors=50)

In [43]:
knn_reg.predict([xm_test[0]])

array([[0.02, 0.14, 0.16, 0.54, 0.76, 0.04, 0.12, 0.36, 0.26]])

In [44]:
ym_test[0] 

array([0., 0., 0., 1., 1., 0., 0., 1., 1.])

### Model 2: Linear Regression

In [45]:
from sklearn.linear_model import LinearRegression
l_reg = LinearRegression()
l_reg.fit(xm_train, ym_train)

LinearRegression()

In [46]:
l_reg.predict([xm_test[7]])

array([[0.21072761, 0.18974704, 0.26153688, 0.17452131, 0.30165461,
        0.21455286, 0.22885301, 0.24238681, 0.26581452]])

In [47]:
ym_test[7]

array([0., 1., 0., 1., 1., 1., 0., 0., 0.])

### Model 3: MLP Regression

In [48]:
from sklearn.neural_network import MLPRegressor
mlp_reg = MLPRegressor(max_iter=2000).fit(xm_train, ym_train)

In [49]:
mlp_reg.predict([xm_test[20]])

array([[-0.06403232, -0.06539061,  0.19832848,  0.03638895, -0.05870963,
        -0.17153422,  0.75692172,  0.45497429,  0.12245563]])

In [50]:
ym_test[20]

array([0., 0., 0., 0., 0., 0., 1., 0., 0.])

### Reporting Performance on Intermediate Datasets:

Measuring Accuracy using the mean squared error:

In [51]:
inter_cvs_svm = cross_val_score(knn_reg, xm_test, ym_test, cv=10, scoring="neg_mean_squared_error") # cv parameter sets the number of KFolds
np.mean(inter_cvs_svm)

-0.12344311614876505

In [52]:
inter_cvs_knn = cross_val_score(l_reg, xm_test, ym_test, cv=10, scoring="neg_mean_squared_error")
np.mean(inter_cvs_knn)

-0.17174807073862505

In [53]:
inter_cvs_mlp = cross_val_score(mlp_reg, xm_test, ym_test, cv=10, scoring="neg_mean_squared_error")

In [54]:
np.mean(inter_cvs_mlp)

-0.0809249488481551

We cannot generate a confusion matrix as we would need to score based on a classification metric. 

In [55]:
# Storing the models:
import pickle
with open('./model_outputs/models/knn_reg_inter.pickle', 'wb') as f:
    pickle.dump(knn_reg, f)
with open('./model_outputs/models/l_reg_inter.pickle', 'wb') as f:
    pickle.dump(l_reg, f)
with open('./model_outputs/models/mlp_reg_inter.pickle', 'wb') as f:
    pickle.dump(mlp_reg, f)

# To Load
# with open('./model_outputs/models/mlp_single.pickle', 'rb') as f:
#    mlp = pickle.load(f)